### **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@retailsalesanalysissanil.dfs.core.windows.net/customers")
df.display()

In [0]:
df = df.withColumn("domains", split(col("email"), "@")[1])
df.display()

In [0]:
df.groupBy("domains").agg(count("*").alias("total_count")).orderBy(col("total_count").desc()).display()

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(' '), col("last_name")))
df = df.drop("first_name", "last_name")
df.display()

### **Data Writing**

In [0]:
df.write.format("delta")\
  .mode("append")\
  .save("abfss://silver@retailsalesanalysissanil.dfs.core.windows.net/customers")

### **Table Creation to top of the Data**

In [0]:
%sql
create table if not exists rsa_cata.silver.customers_silver 
using delta
location 'abfss://silver@retailsalesanalysissanil.dfs.core.windows.net/customers'